<img src="https://raw.githubusercontent.com/datagabe/pics/299e3b5241a8ab9fa8bb6289fc5f66217edf2b19/header.png?raw=true">
This notebook will walk you through the many ways that you can utilize SingleStoreDB today for your AI workloads.



Pre-Reqs:
* Spin up trial on SingleStore cloud at https://portal.singlestore.com
* Create a database order_mgt and load TPCH data. Use this link for instructions - https://docs.singlestore.com/cloud/load-data/load-a-type-of-data/example-load-tpc-h-data-into-singlestoredb-cloud/
* Create your own OPENAI_API_KEY from here - https://platform.openai.com/account/api-keys.
* Create your own SERPAPI_API_KEY from here - https://serpapi.com/users/sign_up

Resources:
* [SingleStore Vector Functions]()
* [Python LangChain](https://python.langchain.com/en/latest/getting_started/getting_started.html)


## Instance variables
These are the defaults for this notebook and should work out of the box for your testing.

## Install Libraries

In [ ]:
!pip install openai singlestoredb langchain langchain_experimental urllib3 tiktoken google-search-results playwright google-api-python-client mongo -q
import os
import requests
from pymongo import MongoClient
from langchain.utilities import SerpAPIWrapper
from langchain import OpenAI, LLMChain
from langchain.sql_database import SQLDatabase
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
#from langchain.experimental import AutoGPT
from langchain_experimental.autonomous_agents import AutoGPT
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import SingleStoreDB, MongoDBAtlasVectorSearch
from langchain.document_loaders import TextLoader, SeleniumURLLoader
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain.memory.chat_memory import ChatMessageHistory
from langchain.memory.chat_message_histories import MongoDBChatMessageHistory
from langchain.docstore import InMemoryDocstore
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent, AgentType, initialize_agent
from langchain.agents.agent_toolkits import FileManagementToolkit, PlayWrightBrowserToolkit
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.tools.playwright.utils import (
    create_async_playwright_browser
)
import ipywidgets as widgets
from IPython.display import display, HTML

# Define the variables
s2_host = connection_host
s2_user = connection_user
s2_pass = connection_password
s2_port = connection_port

os.environ['OPENAI_API_KEY'] = '<Enter you openAI key here'
#os.environ["LANGCHAIN_TRACING"] = "false" # If you want to trace the execution of the program, set to "true"

# Create the interactive widgets
host_widget = widgets.Text(value=s2_host, description='S2 Host:', layout=widgets.Layout(width='auto'))
user_widget = widgets.Text(value=s2_user, description='S2 User:', layout=widgets.Layout(width='auto'))
pass_widget = widgets.Password(value=s2_pass, description='S2 Password:', layout=widgets.Layout(width='auto'))
port_widget = widgets.Text(value=s2_port, description='S2 Port:', layout=widgets.Layout(width='auto'))
url_widget = widgets.Text(description='S2 URL:', layout=widgets.Layout(width='350px'), disabled=True)

# Create helper fields
host_helper = widgets.HTML(value='<em>Enter the S2 host address.</em>', layout=widgets.Layout(color='black'))
user_helper = widgets.HTML(value='<em>Enter the S2 username.</em>', layout=widgets.Layout(color='black'))
pass_helper = widgets.HTML(value='<em>Enter the S2 password.</em>', layout=widgets.Layout(color='black'))
port_helper = widgets.HTML(value='<em>Enter the S2 port number.</em>', layout=widgets.Layout(color='black'))

# Create the update button
update_button = widgets.Button(description='Update', button_style='success')
update_button.style.button_color = '#AA00FF'

# Define the update function
def update_variables(_):
    global s2_host, s2_user, s2_pass, s2_port
    s2_host = host_widget.value
    s2_user = user_widget.value
    s2_pass = pass_widget.value
    s2_port = port_widget.value
    url_widget.value = f'mysql+pymysql://{s2_user}:{s2_pass}@{s2_host}:{s2_port}/'

# Attach the update function to the button's on-click event
update_button.on_click(update_variables)
update_variables('')

#display(HTML(f'<style>{style}</style>'))

# Display the widgets and the button in a vertical layout
widgets.VBox([
    widgets.HBox([host_widget, host_helper]),
    widgets.HBox([user_widget, user_helper]),
    widgets.HBox([pass_widget, pass_helper]),
    widgets.HBox([port_widget, port_helper]),
    widgets.HBox([url_widget]),
    widgets.HBox([update_button])
])


## Chatting over documents with SingleStoreDB

<img src="https://github.com/datagabe/pics/blob/5bec4a32827927e79e531df934417c2653d52e8d/vectorize.png?raw=true" width="1000">

In [ ]:
# Google Search Plugin
#search = GoogleSearchAPIWrapper()
from langchain.agents import load_tools
# Create your own SERPAPI_API_KEY from here - https://serpapi.com/users/sign_up
os.environ['SERPAPI_API_KEY'] = '<Your SERPAPI_API_KEY>'
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(root_dir='/tmp/demo'),
    ReadFileTool(root_dir='/tmp/demo'),
]
# Setup SingleStoreDB Database
%sql create database if not exists vectorSearch
%sql create database if not exists autogpt

os.environ['SINGLESTOREDB_URL'] = f'admin:SingleStore1!@{s2_host}:{s2_port}'

# Inserting documents into SingleStoreDB
r = requests.get('https://raw.githubusercontent.com/hwchase17/langchain/2c91f0d750eb153f72f7d095fadde18f8c683de8/docs/modules/state_of_the_union.txt')
open('/tmp/state_of_the_union.txt', 'wb').write(r.content)

loader = TextLoader('/tmp/state_of_the_union.txt')
#loader = SeleniumURLLoader(urls=urls)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

docsearch = SingleStoreDB.from_documents(
    docs,
    embeddings,
    table_name = "vectorSearch.stateOfTheUnion"
)

In [ ]:
# Prompting SingleStoreDB for similarity with OpenAI embeddings
query = "What did the president say about Ketanji Brown Jackson"
#query = "What did the president say about COVID-19"
docs = docsearch.similarity_search(query) # Find documents that correspond to the query
print(docs[0].page_content)

## Answering questions about your data in SingleStoreDB with OpenAI

### Connecting an OpenAI Agent to your SingleStore Data

In [ ]:
db = SQLDatabase.from_uri(f"mysql+pymysql://admin:SingleStore1!@{s2_host}:3306/order_mgt")
llm = OpenAI(temperature=0, verbose=False)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True
)

In [ ]:
agent_executor.run("Write a SingleStore query that shows the highest paying customers per country, include how much they've spent, use the nation name")